# Clase 8 (RAG) – Versión 100% Open Source

Stack: Ollama (Qwen 2.5 7B Instruct) + FAISS + HuggingFace Embeddings.
Fuente: `PDF/9587014499.PDF` dentro del repo.

In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv; load_dotenv()

from langchain_ollama import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
# Import actualizado (evita deprecación)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

REPO_ROOT = Path.cwd().parents[0] if (Path.cwd()/"notebooks").exists() else Path.cwd()
PDF_PATH = REPO_ROOT / "PDF" / "9587014499.PDF"
INDEX_DIR = REPO_ROOT / ".rag_index" / "faiss-e5-small"

MODEL = os.getenv("MODEL", "qwen2.5:7b-instruct")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
EMB_MODEL = os.getenv("EMB_MODEL_NAME", "intfloat/multilingual-e5-small")

CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "900"))
CHUNK_OVERLAP = int(os.getenv("CHUNK_OVERLAP", "150"))
TOP_K = int(os.getenv("TOP_K", "4"))

# Fuerza embeddings en CPU para evitar conflicto con Ollama usando GPU
EMB = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

LLM = ChatOllama(model=MODEL, base_url=OLLAMA_BASE_URL, temperature=0.2)

INDEX_DIR.mkdir(parents=True, exist_ok=True)
if (INDEX_DIR/"index.faiss").exists() and (INDEX_DIR/"index.pkl").exists():
    vs = FAISS.load_local(INDEX_DIR, EMB, allow_dangerous_deserialization=True)
else:
    loader = PyPDFLoader(str(PDF_PATH))
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    vs = FAISS.from_documents(chunks, EMB)
    vs.save_local(INDEX_DIR)

retriever = vs.as_retriever(search_kwargs={"k": TOP_K})

system = (
    "Eres un asistente que responde SOLO con el contexto del PDF. "
    "Si no está en el PDF, dilo claramente. Responde en español."
)
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "Pregunta: {question}\n\nContexto:\n{context}\n\nRespuesta concisa y precisa:")
])

def format_docs(docs):
    out = []
    for i, d in enumerate(docs, 1):
        pg = (d.metadata or {}).get("page", "?")
        out.append(f"[{i}] (pág. {pg}) {d.page_content.strip()}")
    return "\n\n".join(out)

chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt | LLM | StrOutputParser())

print("RAG listo. PDF:", PDF_PATH)

RAG listo. PDF: /home/wilson/Documentos/my_course_agent/notebooks/PDF/9587014499.PDF


In [4]:
q = "Resume en 5 viñetas los puntos principales del documento."
resp = chain.invoke(q)
print(resp)

- Se calculan probabilidades condicionales y conjuntas para eventos V e E, resultando en P(V I E) = 5/12, P(E I V) = 4/9 y P(V) = 4/3.
- Se presentan respuestas a ejercicios de probabilidad y estadística, incluyendo cálculos de probabilidades condicionales, esperanzas matemáticas y variables aleatorias.
- Se discuten fórmulas para la expectación condicional y varianza en funciones de varias variables, junto con ejemplos específicos.
- Se proporciona una demostración matemática sobre el comportamiento límite de funciones de distribución acumulativa en un espacio bidimensional.


## Buenas prácticas
- Cambia `EMB_MODEL` si quieres otro embedding open-source (p. ej., `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`).
- Sube/edita más PDFs y genera índices separados en `.rag_index/`.
- Integra la herramienta `rag_search` en tu grafo para usar RAG bajo demanda.